In [ ]:
pip install rapidfuzz

In [ ]:
import json
from sklearn.metrics import precision_recall_fscore_support

#without fuzz
with open(r"C:\Users\bauke\llama_results3.json", encoding="utf-8") as f:
    predictions = json.load(f)

with open(r"C:\Users\bauke\OneDrive - KU Leuven\Documents\Documenten\5 digital humanities\stage\Marcel150_clean.json", encoding="utf-8") as f:
    annotated = json.load(f)

gold_entities = {}
for item in annotated:
    filename = item["data"]["filename"]
    entities = item["annotations"][0]["result"]
    gold_per = set()
    gold_loc = set()
    for ent in entities:
        label = ent["value"]["labels"][0]
        text = ent["value"]["text"].strip()
        if label == "PER":
            gold_per.add(text)
        elif label == "LOC":
            gold_loc.add(text)
    gold_entities[filename] = {"PER": gold_per, "LOC": gold_loc}

y_true = []
y_pred = []

for article in predictions:
    filename = article["filename"]
    predicted_entities = article.get("entities", [])

    pred_per = set(ent["text"].strip() for ent in predicted_entities if ent["label"] == "PER")
    pred_loc = set(ent["text"].strip() for ent in predicted_entities if ent["label"] == "LOC")

    gold = gold_entities.get(filename, {"PER": set(), "LOC": set()})
    gold_per = gold["PER"]
    gold_loc = gold["LOC"]

    for ent in pred_per:
        y_pred.append("PER")
        y_true.append("PER" if ent in gold_per else "O")
    for ent in gold_per:
        if ent not in pred_per:
            y_pred.append("O")
            y_true.append("PER")

    for ent in pred_loc:
        y_pred.append("LOC")
        y_true.append("LOC" if ent in gold_loc else "O")
    for ent in gold_loc:
        if ent not in pred_loc:
            y_pred.append("O")
            y_true.append("LOC")

labels = ["PER", "LOC"]
precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred, labels=labels, zero_division=0)

print("\nEntity-Level Evaluation Report (NER format with offsets):")
for i, label in enumerate(labels):
    print(f"{label}:")
    print(f"  Precision: {precision[i]:.4f}")
    print(f"  Recall:    {recall[i]:.4f}")
    print(f"  F1 Score:  {f1[i]:.4f}")
    print(f"  Support:   {support[i]}")


In [ ]:
import json
from rapidfuzz import fuzz
from sklearn.metrics import precision_recall_fscore_support

#with fuzz

FUZZY_THRESHOLD = 60  

def is_fuzzy_match(pred, gold_set, threshold=FUZZY_THRESHOLD):
    return any(fuzz.ratio(pred, gold) >= threshold for gold in gold_set)

with open(r"C:\Users\bauke\ner_results_trainedformllm.json", encoding="utf-8") as f:
    predictions = json.load(f)

with open(r"C:\Users\bauke\OneDrive - KU Leuven\Documents\Documenten\5 digital humanities\stage\Marcel150_clean.json", encoding="utf-8") as f:
    annotated = json.load(f)

gold_entities = {}
for item in annotated:
    filename = item["data"]["filename"]
    entities = item["annotations"][0]["result"]
    gold_per = set()
    gold_loc = set()
    for ent in entities:
        label = ent["value"]["labels"][0]
        text = ent["value"]["text"].strip()
        if label == "PER":
            gold_per.add(text)
        elif label == "LOC":
            gold_loc.add(text)
    gold_entities[filename] = {"PER": gold_per, "LOC": gold_loc}

y_true = []
y_pred = []

for article in predictions:
    filename = article["filename"]
    predicted_entities = article.get("entities", [])

    pred_per = set(ent["text"].strip() for ent in predicted_entities if ent["label"] == "PER")
    pred_loc = set(ent["text"].strip() for ent in predicted_entities if ent["label"] == "LOC")

    gold = gold_entities.get(filename, {"PER": set(), "LOC": set()})
    gold_per = gold["PER"]
    gold_loc = gold["LOC"]

    matched_gold_per = set()
    matched_gold_loc = set()

    for pred in pred_per:
        if is_fuzzy_match(pred, gold_per):
            y_pred.append("PER")
            y_true.append("PER")
            matched_gold_per.update({g for g in gold_per if fuzz.ratio(pred, g) >= FUZZY_THRESHOLD})
        else:
            y_pred.append("PER")
            y_true.append("O")
    for gold in gold_per:
        if gold not in matched_gold_per:
            y_pred.append("O")
            y_true.append("PER")

    for pred in pred_loc:
        if is_fuzzy_match(pred, gold_loc):
            y_pred.append("LOC")
            y_true.append("LOC")
            matched_gold_loc.update({g for g in gold_loc if fuzz.ratio(pred, g) >= FUZZY_THRESHOLD})
        else:
            y_pred.append("LOC")
            y_true.append("O")
    for gold in gold_loc:
        if gold not in matched_gold_loc:
            y_pred.append("O")
            y_true.append("LOC")

labels = ["PER", "LOC"]
precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred, labels=labels, zero_division=0)

print("\nEntity-Level Evaluation Report (Fuzzy Matching):")
for i, label in enumerate(labels):
    print(f"{label}:")
    print(f"  Precision: {precision[i]:.4f}")
    print(f"  Recall:    {recall[i]:.4f}")
    print(f"  F1 Score:  {f1[i]:.4f}")
    print(f"  Support:   {support[i]}")
